In [284]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Make some activation functions for the hell of it

In [285]:
# Lets get some activation functions going:

softmax = lambda x : np.exp(x) / np.sum(np.exp(x), axis=0)

sigmoid = lambda x : 1/(1+np.exp(-x))

sigmoid_derivative = lambda x : sigmoid(x) * (1 - sigmoid(x))

relu = lambda x : np.maximum(0,x)

identity = lambda x : x

one = lambda x : 1

# Defining a layer

In [286]:

class Layer():
    def __init__(self, width, activation=identity):
        self.width = width
        self.activation = activation
        self.weights = None
        self.bias = None
        self.input_width = None
        
    def construct(self, input_width):
        # input_width is the width of the previous layer
        if self.input_width != None:
            raise AttributeError("Already initialised")
        else:
            self.input_width = input_width
            self.weights = np.random.rand(self.width, self.input_width)
            self.bias = np.random.rand()
            
    def eval(self,input_values):
        if self.input_width == None:
            raise AttributeError("Not initialised")
        else:
            return self.activation((self.weights @ input_values) + self.bias)


# Defining a Neural Network in terms of layers

In [287]:
square_cost_derivative = lambda x,y : x - y
square_cost = lambda x,y : (x - y) * (x - y) / 2

class NeuralNetwork:
    def __init__(self, sizes, act=sigmoid, act_deriv=sigmoid_derivative, cost=square_cost, cost_derivative=square_cost_derivative):
        """
        sizes:list of the sizes of layers
        """
        self.n_layers = len(sizes)
        self.sizes = sizes
        
        # Don't need biases for inputs so we do sizes[1:]
        self.biases = [np.random.randn(size) for size in sizes[1:]]
        
        # Need weights between the layers.
        # n x m matrix where n is # of perceptrons on output layer
        # and m is # of perceptrons on input layer
        self.weights = [np.random.randn(n_out,n_in) for n_in,n_out in zip(sizes[:-1],sizes[1:])]
        
        self.activation = act
        self.activation_derivative = act_deriv
        self.cost = cost
        self.cost_derivative = cost_derivative
        
    def ff(self, input_values):
        for weight, bias in zip(self.weights,self.biases):
            print("weights: ", weight.shape, " biases: ", bias.shape, " input: ", input_values.shape)
            input_values = self.activation(weight @ input_values + bias)
            print(input_values)
        return input_values
    
    def SGD(self, training_set, epochs=1, mini_batch_size=1, learning_rate=1):
        for i in range(epochs):
            mini_batches = self.split_training_set(training_set,mini_batch_size)
            
            for mb in mini_batches:
                self.update_mini_batch(mb, learning_rate)
                
    def update_mini_batch(self, mini_batch, learning_rate):
        change_biases = [np.zeros(bias.shape) for bias in self.biases]
        change_weights = [np.zeros(weight.shape) for weight in self.weights]
        
        for x,y in mini_batch:
            delta_biases, delta_weights = self.backprop(x,y)
            change_biases = [old + d for old,d in zip(change_biases, delta_biases)]
            change_weights = [old + d for old,d in zip(change_weights, delta_weights)]
        self.weights = [w + (learning_rate/len(mini_batch)) * cw for w,cw in zip(self.weights,change_weights)]
        self.biases = [b + (learning_rate/len(mini_batch)) * cb for b,cb in zip(self.biases,change_biases)]
        
    def backprop(self,x,y):
        # x is vector for the inputs
        # y is also a vector but for the outputs
        
        activation = x
        activations = [activation]
        nets = []
        
        # feed forward and store activations
        
        for w,b in zip(self.weights,self.biases):
            net = w @ activation + b
            nets.append(net)
            
            activation = self.activation(net)
            activations.append(activation)
            
        # backwards pass
        delta = self.cost_derivative(activations[-1],y) * self.activation_derivative(activation)
        
        change_b = [np.zeros(b.shape) for b in self.biases]
        change_w = [np.zeros(w.shape) for w in self.weights]
        
        #print("delta:",delta,delta.reshape(-1,1).shape)
        #print("activations:",activations[-2].shape)
    
                        
        change_b[-1] = delta
        change_w[-1] = delta.reshape(-1,1) @ activations[-2].reshape(1,-1)
        
        for i in range(2,self.n_layers):
            net = nets[-i]
            delta = (self.weights[-i + 1].T @ delta) * self.activation_derivative(net)
            change_b[-i] = delta
            change_w[-i] = delta.reshape(-1,1) @ activations[-i-1].reshape(1,-1)
            
        return change_b,change_w
    
    def split_training_set(self,training_set,mini_batch_size):
        np.random.shuffle(training_set)
        return [training_set[k:k+mini_batch_size] for k in range(0, len(training_set), mini_batch_size)]
            
            
            
    
    def __str__(self):
        s = []
        s.append(" ".join([f"input({n+1})" for n in range(self.sizes[0])]))
        for weight,bias in zip(self.weights,self.biases):
            s.append("   |")
            s.append("   \\/")
            s.append(np.array2string(weight) + "  +  " + np.array2string(bias))
        return "\n".join(s)
            
        

# Testing

In [288]:
nn = NeuralNetwork([2,3,2])
print(nn)


input(1) input(2)
   |
   \/
[[ 0.0622238   0.71263721]
 [ 0.39559325 -0.49083747]
 [-0.20359484  1.84865939]]  +  [-1.41275375 -1.00388875 -1.6361178 ]
   |
   \/
[[-0.34371938  0.37437389  0.34540983]
 [ 0.02119479 -0.43360789 -0.2905119 ]]  +  [-0.45860111  0.51439198]


In [289]:
nn.ff(np.array([0,0]))

weights:  (3, 2)  biases:  (3,)  input:  (2,)
[0.19580008 0.26817753 0.16299401]
weights:  (2, 3)  biases:  (2,)  input:  (3,)
[0.40873362 0.58780684]


array([0.40873362, 0.58780684])

In [290]:
training_set = [(np.array([1,1]),np.array([1,1])),
               (np.array([1,3]),np.array([1,3])),
               (np.array([2,1]),np.array([2,1])),
               (np.array([0,1]),np.array([0,1])),]

In [291]:
nn.SGD(training_set,epochs=100)

In [292]:
nn.ff(np.array([1,5]))

weights:  (3, 2)  biases:  (3,)  input:  (2,)
[1. 1. 1.]
weights:  (2, 3)  biases:  (2,)  input:  (3,)
[4.31711374e-172 4.56994322e-258]


array([4.31711374e-172, 4.56994322e-258])

In [293]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs=1, mini_batch_size=1, eta=1,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [294]:
nn1 = Network([2,3,2])
test_data = [(np.array([0,0]), np.array([0,0]))]
nn1.SGD(training_set,test_data=test_data)

NameError: name 'xrange' is not defined